In [1]:
import pandas as pd

In [2]:
## MAIN FUNCTION - input

## INPUT FILE
df_input_female = pd.read_csv('BL2013_F_v2.2.csv')
df_input_male = pd.read_csv('BL2013_M_v2.2.csv')
df_project_female = pd.read_csv('OUP_proj_F1564_v1.csv')
df_project_male = pd.read_csv('OUP_proj_M1564_v1.csv')

country_names = pd.read_csv('../../data/dl1_countrycodeorg_country_name.csv')
codemap = dict(zip(country_names['Country Code'], country_names.country))
WBcodes = sorted(df_input_female['WBcode'].unique()) # 146

## OUTPUT FILE
save_file_name = "education.csv"
save_filled_file_name = "education_filled.csv"
save_check_name = 'check_country_name.csv'

In [3]:
def transage(x):
    if x<65:
        return 'd'+str(x)
    else:
        return 'd65'

In [4]:
def check_education_country_name(df_project_female, country_names):
    print ('Not included WBcodes, Please check')
    print(set(df_project_female['WBcode'].unique()) - set(country_names['Country Code'].unique()))
    # SER SRB, Serbia 
    # ROM ROU, Romania
    Country_Codes_edu = df_project_female[['WBcode', 'country']].drop_duplicates()
    Country_Codes_edu.columns = ['Country Code', 'country_edu']
    Country_Codes_names = country_names[['Country Code', 'country']].drop_duplicates()
    checkfile = pd.merge(Country_Codes_edu, Country_Codes_names, left_on= 'Country Code', right_on='Country Code')

    checkfile.to_csv(save_check_name)
    print ('The country names with same WBcodes are saved in check_country_name.csv file, Please check')

In [5]:
def ProcessALL(df_input, df_project):
    pieces = []
    for WBcode in WBcodes:
        if WBcode == 'SER':
            country_code = 'SRB'
            print ('WBcode SER -> Country Code SRB')
        elif WBcode == 'ROM':
            country_code = 'ROU'
            print ('WBcode ROM -> Country Code ROU')   
        else:
            country_code = WBcode  
        print(WBcode)
        data = Process(WBcode, df_input, df_project)
        data['Country Code'] = country_code 
        pieces.append(data)
    return pieces

In [6]:
def Process(WBcode, df_input, df_project):
    df = df_input[(df_input['WBcode']==WBcode)&(df_input['year']>=1990)]
    df = df[['year', 'sex', 'agefrom', 'ageto', 'yr_sch', 'pop']]
    df = df[~((df['agefrom']<75)&(df['ageto']==999))] # delete 15+ and 25+
    df['age'] = df['agefrom'].apply(lambda x: transage(x))
    df = df[['age','year','sex','yr_sch', 'pop']]
    df['yr_sch']=df['yr_sch']*df['pop']
    
    ####################### get 15-65 for future projection
    df15 = df[(df['age'] !='d65') & (df['year'] == 2010)]  # 15-65
    gb15 = df15.groupby(['sex', 'year']).sum()
    gb15['yr_sch']=gb15['yr_sch']/gb15['pop']
    gb15.drop('pop', axis=1, inplace=True) 

    #######################
    dff = df.groupby(['age', 'sex', 'year']).sum()
    dff['yr_sch']=dff['yr_sch']/dff['pop']
    dff.reset_index(inplace=True)
    out1 = dff[['age','year','sex','yr_sch']]

    ## get projection scales
    df_pro = df_project[df_project['WBcode']==WBcode][['sex', 'year', 'yr_sch']]
    df_pro = df_pro.set_index(['year'])[['yr_sch']]
    df_pro = df_pro.div(gb15.iloc[0, 0]).transpose()
    dff = pd.pivot_table(out1, index=['year'], columns=['sex', 'age'], values='yr_sch').transpose().reset_index()

    for year in [2015,2020,2025,2030, 2035, 2040]:
        dff[year]=dff[2010]*df_pro.at['yr_sch',year]
    dff = dff.set_index(['sex', 'age']).transpose()
    years = [1990 + i for i in range(51)] # interpolate
    df2 = dff.reindex(years)
    df2 = df2.apply(pd.Series.interpolate)
    dff = df2.transpose()
    exp = dff[2040] / dff[2035]
    exp = exp.apply(lambda x: pow(x, 1 / 5.0))
    for year in range(2041, 2051, 1):
        dff[year] = dff[year - 1] * exp
    out = dff.fillna(0)
    return out


In [7]:
## Check Countries
# check_education_country_name(df_input_female, country_names)
# We need to change SER to SRB, ROM to ROU
## MAIN FUNCTION - output

df_input_female = df_input_female[['WBcode', 'year', 'sex', 'agefrom', 'ageto', 'yr_sch', 'pop']]
df_project_female = df_project_female[['WBcode', 'sex', 'year', 'yr_sch']]
pieces1 = ProcessALL(df_input_female, df_project_female)

df_input_male = df_input_male[['WBcode', 'year', 'sex', 'agefrom', 'ageto', 'yr_sch', 'pop']]
df_project_male = df_project_male[['WBcode', 'sex', 'year', 'yr_sch']]
pieces2 = ProcessALL(df_input_male, df_project_male)

primary = pd.concat(pieces1 + pieces2)

education = primary.reset_index().set_index('Country Code').round(3)
education.to_csv(save_file_name)

AFG
ALB
ARE
ARG
ARM
AUS
AUT
BDI
BEL
BEN
BGD
BGR
BHR
BLZ
BOL
BRA
BRB
BRN
BWA
CAF
CAN
CHE
CHL
CHN
CIV
CMR
COD
COG
COL
CRI
CUB
CYP
CZE
DEU
DNK
DOM
DZA
ECU
EGY
ESP
EST
FIN
FJI
FRA
GAB
GBR
GHA
GMB
GRC
GTM
GUY
HKG
HND
HRV
HTI
HUN
IDN
IND
IRL
IRN
IRQ
ISL
ISR
ITA
JAM
JOR
JPN
KAZ
KEN
KGZ
KHM
KOR
KWT
LAO
LBR
LBY
LKA
LSO
LTU
LUX
LVA
MAC
MAR
MDV
MEX
MLI
MLT
MMR
MNG
MOZ
MRT
MUS
MWI
MYS
NAM
NER
NIC
NLD
NOR
NPL
NZL
PAK
PAN
PER
PHL
PNG
POL
PRT
PRY
QAT
REU
WBcode ROM -> Country Code ROU
ROM
ROU
RUS
RWA
SAU
SDN
SEN
WBcode SER -> Country Code SRB
SER
SGP
SLE
SLV
SVK
SVN
SWE
SWZ
SYR
TGO
THA
TJK
TON
TTO
TUN
TUR
TWN
TZA
UGA
UKR
URY
USA
VEN
VNM
YEM
ZAF
ZMB
ZWE
AFG
ALB
ARE
ARG
ARM
AUS
AUT
BDI
BEL
BEN
BGD
BGR
BHR
BLZ
BOL
BRA
BRB
BRN
BWA
CAF
CAN
CHE
CHL
CHN
CIV
CMR
COD
COG
COL
CRI
CUB
CYP
CZE
DEU
DNK
DOM
DZA
ECU
EGY
ESP
EST
FIN
FJI
FRA
GAB
GBR
GHA
GMB
GRC
GTM
GUY
HKG
HND
HRV
HTI
HUN
IDN
IND
IRL
IRN
IRQ
ISL
ISR
ITA
JAM
JOR
JPN
KAZ
KEN
KGZ
KHM
KOR
KWT
LAO
LBR
LBY
LKA
LSO
LTU
LUX
LVA
MAC
MAR
MDV
MEX
MLI
MLT
MMR
MN

In [8]:
df = education.merge(country_names[['Country Code','Region','Income group']],on='Country Code')
ave = df.groupby(['sex','age','Region','Income group']).mean().reset_index()
tofill = country_names[~country_names['Country Code'].isin(df['Country Code'])][['Country Code','Region','Income group']]
filled = tofill.merge(ave, on=['Region','Income group'])
data = pd.concat([df, filled])
data = data.drop(['Region','Income group'],axis = 1)
data.set_index(['Country Code','sex','age']).to_csv(save_filled_file_name)

## Check data, summary

In [9]:
def check_countries(filename):
    df = pd.read_csv(filename)
    countries = df['Country Code'].unique()
    country_names = pd.read_csv('../../data/dl1_countrycodeorg_country_name.csv')
    GBD_countries = country_names[country_names["country"].notnull()]['Country Code']
    WB_countries = country_names[country_names["WBCountry"].notnull()]['Country Code']
    plus_GBD = sorted(list(set(countries) - set(GBD_countries)))
    sub_GBD = sorted(list(set(GBD_countries) - set(countries)))
    plus_WB = sorted(list(set(countries) - set(WB_countries)))
    sub_WB = sorted(list(set(WB_countries) - set(countries)))

    print ('Subtraction from WB:', sub_WB, len(sub_WB))
    print ('Plus from WB:', plus_WB, len(plus_WB))
    print ('Subtraction from GBD :', sub_GBD, len(sub_GBD))
    print ('Plus from GBD :', plus_GBD, len(plus_GBD))

In [10]:
check_countries(save_filled_file_name)

print(tofill[tofill['Country Code'] == "SYC"])
t1 = ave[ave['Region'] == 'Sub-Saharan Africa']
t2 = t1[t1['Income group'] == 'High income']
print (t2.shape)

print(tofill[tofill['Country Code'] == "PRK"])
t3 = ave[ave['Region'] == 'East Asia & Pacific']
t4 = t3[t3['Income group'] == 'Low income']
print (t4.shape)

# in GBD, not WBcountry: COK NIU PSE TKL (TWN) 
# not GBD, not WBcountry: AIA ANT ATA BLM CCK CXR ESH FLK GGY IOT JEY MSR MYT PCN SHN SJM SPM VAT WLF (REU)
# in GBD, in WBcountry: PRK  SYC 

Subtraction from WB: ['PRK', 'SYC'] 2
Plus from WB: ['REU', 'TWN'] 2
Subtraction from GBD : ['COK', 'NIU', 'PRK', 'PSE', 'SYC', 'TKL'] 6
Plus from GBD : ['ABW', 'CHI', 'CUW', 'CYM', 'FRO', 'GIB', 'HKG', 'IMN', 'LIE', 'MAC', 'MAF', 'NCL', 'PYF', 'REU', 'SXM', 'TCA', 'VGB', 'XKX'] 18
    Country Code              Region Income group
205          SYC  Sub-Saharan Africa  High income
(0, 65)
    Country Code               Region Income group
174          PRK  East Asia & Pacific   Low income
(0, 65)
